In [1]:
import requests
import csv
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
from numpy import arange

In [2]:
inexp_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [3]:
mod_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [4]:
pricey_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.3&ed_attrs=RestaurantsPriceRange2.1'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can calculate total pages by using the total number of resturants (found in plain html at the top of each price point page) and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant names first and convert to all small case with dashes put between each word and punctuation marks removed in order to construct URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>

In [5]:
#Put price point pages in list
price_points = list()
price_points.append(inexp_page1)
price_points.append(mod_page1)
price_points.append(pricey_page1)

In [6]:
#list for resturants in each price point
pp_rest = list()

In [7]:
#get number of pages of resturants for each price point
for price_point in price_points:
    response = requests.get(price_point)
    content = response.content
    pp_rest.append(re.findall('Showing 1-30 of....',str(content)))

In [8]:
pp_rest

[['Showing 1-30 of 447'], ['Showing 1-30 of 819'], ['Showing 1-30 of 107']]

In [10]:
#dictionary for pages
price_rests = dict()
#extract last 3 numbers
pp_rest[0] = str(pp_rest[0])
rests = re.findall('[0-9]{3}',str(pp_rest[0]))
rests = str(rests)
rests = rests[2:5]
price_rests['inexp'] = int(rests)
#---------------------------------------
pp_rest[1] = str(pp_rest[1])
rests = re.findall('[0-9]{3}',str(pp_rest[1]))
rests = str(rests)
rests = rests[2:5]
price_rests['mod'] = int(rests)
#---------------------------------------
pp_rest[2] = str(pp_rest[2])
rests = re.findall('[0-9]{3}',str(pp_rest[2]))
rests = str(rests)
rests = rests[2:5]
price_rests['pricey'] = int(rests)

In [11]:
price_rests

{'inexp': 447, 'mod': 819, 'pricey': 107}

In [12]:
#calculate number of pages using number of resturants
pp_pages = dict()
for pp in price_rests:
    pages = (price_rests[pp]/30) + 1
    pp_pages[pp] = int(pages)

In [13]:
pp_pages

{'inexp': 15, 'mod': 28, 'pricey': 4}

## 2.0 Extract and Process all resturant names for each price point
### 2.1 Cheapest price point (Inexp)

In [22]:
#first we construct all URLs
inexp_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [23]:
secondpage = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [24]:
#After examining the URLs,'&start=30' is added to the end of the url for page 2 (30 is added for every page)
page2 = inexp_page1 + '&start=30'

In [27]:
#The last page url of any price point is therefore constructed by adding '&start=(MaxPages-1)*30'
page2

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [77]:
#lets now save this as resturant count, page 2 = 30, page 3 = 60....page 15 = 420
resturant_count = list()
#declare another list to keep all scrapped dirty names from the inexp price point
dirty_names = list()

In [78]:
for x in range(1,15):
    resturant_count.append(str((x*30)))

In [79]:
#Get resturant names from each page, start with page 1 then loop through the rest
response = requests.get(inexp_page1)
content = response.content
copycontent = str(content)
dirtynames = re.findall('reviewCount":[0-9],"name":".{23}',copycontent)
dirty_names.append(dirtynames)
for count in resturant_count:
    new_page = inexp_page1 + '&start=' + count
    response = requests.get(new_page)
    content = response.content
    copycontent = str(content)
    dirtynames = list()
    dirtynames = re.findall('reviewCount":[0-9],"name":".{23}',copycontent)
    dirty_names.append(dirtynames)

In [54]:
type(dirtynames)

list

In [58]:
for count in resturant_count:
    print(count)

In [81]:
len(dirty_names[0])

22

In [75]:
inexp_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [82]:
dirty_names[0]

['reviewCount":5,"name":"Frango","neighborhoods"',
 'reviewCount":3,"name":"Aldhiaffah Al-Iraqi Res',
 'reviewCount":2,"name":"Darband Restaurant","ne',
 'reviewCount":2,"name":"Pho Ba Le Vietnamese No',
 'reviewCount":1,"name":"Al Shami - Syrian &amp;',
 'reviewCount":8,"name":"Hawa Charcoal Chicken",',
 'reviewCount":4,"name":"Awafi Charcoal Chicken"',
 'reviewCount":4,"name":"Gourmet Noodle House","',
 'reviewCount":8,"name":"Pho Sam Noodle House","',
 'reviewCount":6,"name":"El Jannah","neighborhoo',
 'reviewCount":7,"name":"Pho Mai","neighborhoods',
 'reviewCount":8,"name":"New Star Kebabs","neigh',
 'reviewCount":3,"name":"Banh Cuon Kim Thanh","n',
 'reviewCount":8,"name":"El Burger","neighborhoo',
 'reviewCount":2,"name":"Mr Happy Chef Noodle Ho',
 'reviewCount":3,"name":"Chubby Buns Burger","ne',
 'reviewCount":6,"name":"Mr Ping\\xe2\\x80\\x99s","',
 'reviewCount":7,"name":"Pho Saigon","neighborho',
 'reviewCount":3,"name":"Lucky Thai &amp; Lao Re',
 'reviewCount":8,"name":"Kna

In [87]:
suma = 0
for name in dirty_names:
    suma = suma + len(name)
    print(len(name))

22
27
30
30
28
27
28
30
30
30
30
30
30
30
27


In [88]:
suma

429